In [1]:
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, DataCollatorForWholeWordMask

In [2]:
mlm_probability = 0.15
pad_to_max_length = False

In [3]:
data_files = {}
data_files['train'] = 'asr-text.txt'

In [4]:
extension = 'text'
datasets = load_dataset(extension, data_files=data_files)

Using custom data configuration default-01d78f19638e80c5


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /home/ubuntu/.cache/huggingface/datasets/text/default-01d78f19638e80c5/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained('./malay-cased-bert-base')

In [6]:
column_names = datasets["train"].column_names
text_column_name = "text" if "text" in column_names else column_names[0]
padding = "max_length" if pad_to_max_length else False

In [7]:
max_seq_length = 256

def tokenize_function(examples):
    # Remove empty lines
    examples["text"] = [line for line in examples["text"] if len(line) > 0 and not line.isspace()]
    return tokenizer(examples["text"], padding=padding, truncation=True, max_length=max_seq_length)

In [8]:
overwrite_cache = False
tokenized_datasets = datasets.map(
    tokenize_function,
    batched=True,
    num_proc=1,
    remove_columns=[text_column_name],
    load_from_cache_file=not overwrite_cache,
)

  0%|          | 0/1405 [00:00<?, ?ba/s]

In [9]:
data_collator = DataCollatorForWholeWordMask(tokenizer=tokenizer, mlm_probability=0.15)

In [10]:
import numpy as np

In [14]:
index = 10
r = data_collator([tokenized_datasets['train'][index]])
r

{'input_ids': tensor([[   2,    4, 3217, 1296,    3]]),
 'labels': tensor([[-100, 2622, -100, -100, -100]])}

In [16]:
tokenizer.decode(r['input_ids'][0].detach().numpy())

'[CLS] [MASK] tanya aku [SEP]'

In [18]:
tokenizer.decode(tokenized_datasets['train'][index]['input_ids'])

'[CLS] jangan tanya aku [SEP]'